In [1]:
#Define the network
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
#The learnable parameters of a model are returned by net.parameters()
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [3]:
#Let’s try a random 32x32 input,'processing inputs'
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1050, -0.1294, -0.0300, -0.1684,  0.0915, -0.0583, -0.1495, -0.0190,
          0.0895,  0.1536]], grad_fn=<AddmmBackward0>)


In [4]:
#Zero the gradient buffers of all parameters and backprops with random gradients
net.zero_grad()  #gradient buffer
#'calling backward'
out.backward(torch.randn(1, 10)) #para

In [5]:
##torch.nn only supports mini-batches. The entire torch.nn package only supports inputs that are a mini-batch of samples, and not a single sample.

#For example, nn.Conv2d will take in a 4D Tensor of nSamples x nChannels x Height x Width.

#If you have a single sample, just use input.unsqueeze(0) to add a fake batch dimension.

In [6]:
#Loss Function
output = net(input)
target = torch.randn(10)  # a dummy target, for example
#print(target)
target = target.view(1, -1)  # make it the same shape as output,参数中的-1就代表这个位置由其他位置的数字来推断
criterion = nn.MSELoss() #mean-squared error between the output and the target
#print(target)
loss = criterion(output, target)
print(loss)

tensor(0.3833, grad_fn=<MseLossBackward0>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [8]:
#Backprop
#You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0024, -0.0013, -0.0021, -0.0036,  0.0024, -0.0068])


In [10]:
##
#Update the weights
#The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):
# the form is :weight = weight - learning_rate * gradient

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [11]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update